First, create the model. This must match the model used in the interactive training notebook.

In [ ]:
import torch
import torchvision

CATEGORIES = ['blocked', 'free']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [ ]:
model.load_state_dict(torch.load('collision_model.pth'))

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [ ]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [ ]:
torch.save(model_trt.state_dict(), 'blocked_free_model_trt.pth')

Load the optimized model by executing the cell below

In [ ]:
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('blocked_free_model_trt.pth'))

Create the racecar class

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=60)

Finally, execute the cell below to make the racecar move forward as long as it thinks it is free.

In [ ]:
from utils import preprocess

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    
    if output[CATEGORIES.index('blocked')] > 0.5:
        car.throttle = -1.0 # applying and holding negative value will brake car
    else:
        car.throttle = 0.2